In [1]:
import sys, os, json
from time import time, strftime, localtime
import cPickle as pickle
from ModelExRecCNOT import *
from ModelSurface1EC import *
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
with open('../../Param/PureError/Steane_CNOT_D5/2017-12-18-14-10-08.json') as paramfile:
    param = json.load(paramfile)

In [3]:
param

{u'data': {u'num trials': 10, u'test fraction': 0.1},
 u'env': {u'EC scheme': u'ColorD5',
  u'FT scheme': u'ExRecCNOT',
  u'look up': False,
  u'param folder': u'../../Param/PureError/Steane_CNOT_D5/',
  u'pickle folder': u'../../Data/Pkl/PureError/Steane_CNOT_D5/e-03-2/',
  u'raw folder': u'../../Data/Compact/Steane_CNOT_D5/e-03-2/',
  u'report folder': u'../../Reports/PureError/Steane_CNOT_D5/'},
 u'nn': {u'W std': 0.06402113166946315,
  u'activations': [u'relu', u'id'],
  u'b std': 0.0,
  u'iso': False,
  u'keep rate': 1.0,
  u'mixed': False,
  u'num hidden': [941],
  u'type': u'FF'},
 u'opt': {u'batch size': 2000,
  u'decay': 0.9241752109242891,
  u'iterations': 20,
  u'learning rate': 0.0006542213414571715,
  u'momentum': 0.6464972901885642,
  u'tune value': 5.46169829402979e-05},
 u'usr': {u'verbose': False}}

In [4]:
if(param['env']['EC scheme']=='SurfaceD3'):
    import _SurfaceD3Lookup as lookup 
elif(param['env']['EC scheme']=='SurfaceD5'):
    import _SurfaceD5Lookup as lookup 
elif(param['env']['EC scheme']=='ColorD3'):
    import _ColorD3Lookup as lookup 
elif(param['env']['EC scheme']=='ColorD5'):
    import _ColorD5Lookup as lookup 
else:
    raise ValueError('Unknown circuit type.')
spec= lookup.Spec()

In [5]:
with open('../../Data/Pkl/PureError/Steane_CNOT_D5/e-03/SyndromeAndError2.000e-03.pkl', 'rb') as input_file:
    m = pickle.load(input_file)

m.test_size= int(param['data']['test fraction'] * m.data_size)
m.train_size= m.data_size - m.test_size
m.num_batches= m.train_size // param['opt']['batch size']
m.spec= spec

In [8]:
n= []
with open('../../Data/Pkl/PureError/Steane_CNOT_D5/e-04/SyndromeAndError6.000e-04.pkl', 'rb') as input_file:
    n.append(pickle.load(input_file))
with open('../../Data/Pkl/PureError/Steane_CNOT_D5/e-04/SyndromeAndError7.000e-04.pkl', 'rb') as input_file:
    n.append(pickle.load(input_file))
with open('../../Data/Pkl/PureError/Steane_CNOT_D5/e-04/SyndromeAndError8.000e-04.pkl', 'rb') as input_file:
    n.append(pickle.load(input_file))
with open('../../Data/Pkl/PureError/Steane_CNOT_D5/e-04/SyndromeAndError9.000e-04.pkl', 'rb') as input_file:
    n.append(pickle.load(input_file))
for i in range(len(n)):
    n[i].test_size= int(param['data']['test fraction'] * n[i].data_size)

In [12]:
vals= {}
for i in range(len(n)):
    vals[i]= []

In [9]:
verbose= param['usr']['verbose']
batch_size= param['opt']['batch size']
learning_rate= param['opt']['learning rate']
num_iterations= param['opt']['iterations']
momentum_val= param['opt']['momentum']
decay_rate= param['opt']['decay']

tf.reset_default_graph()
x, y, predict= {}, {}, {}
for key in m.spec.err_keys:
    with tf.variable_scope(key):
        x[key] = tf.placeholder(tf.float32, [None,m.spec.input_size])
        y[key] = tf.placeholder(tf.float32, [None,2])
keep_rate= tf.placeholder(tf.float32)
#cost= surface_conv3d_cost(param['nn'], m.spec, x, y, predict, keep_rate)
cost= m.cost_function(param['nn'], x, y, predict, keep_rate)
train = tf.train.RMSPropOptimizer(learning_rate, decay=decay_rate, \
    momentum=momentum_val).minimize(cost)
init = tf.global_variables_initializer()

In [17]:
for T in range(9):

    pointer= randint(0, m.data_size - 1) # self.test_size * trial
    t_beg= (m.train_size + pointer) % m.data_size

    predictions= []

    with tf.Session() as session:
        session.run(init)

        for i in tqdm.tqdm(range(num_iterations)):
            for j in range(m.num_batches):
                beg= (j * batch_size + pointer) % m.data_size
                feed_dict={}
                for key in m.spec.err_keys:
                    feed_dict[x[key]]= \
                        cyc_pick(m.syn[key], beg, batch_size)
                    feed_dict[y[key]]= \
                        cyc_pick(m.log_1hot[key], beg, batch_size)
                    feed_dict[keep_rate]= param['nn']['keep rate']
                session.run(train, feed_dict)

        for i in tqdm.tqdm(range(len(n))):
            prediction={}
            feed_dict={}
            for key in m.spec.err_keys:
                feed_dict[x[key]]= \
                    cyc_pick(n[i].syn[key], t_beg, n[i].test_size)
            feed_dict[keep_rate]= 1.0
            for key in m.spec.err_keys:
                prediction[key] = session.run(predict[key], feed_dict)
            predictions.append(prediction)

    for i in tqdm.tqdm(range(len(n))):
        vals[i].append(n[i].error_scale * n[i].num_logical_fault(predictions[i], t_beg))
    
    vals

100%|██████████| 4/4 [04:37<00:00, 69.37s/it]


In [18]:
vals

{0: [5.9954728878720084e-05,
  6.974325604259275e-05,
  5.9954728878720084e-05,
  8.687317857936993e-05,
  6.851969014710868e-05,
  8.809674447485402e-05,
  6.607255835614051e-05,
  7.953178320646543e-05,
  7.219038783356092e-05,
  6.362542656517235e-05],
 1: [0.00011493618267516545,
  9.531293197452746e-05,
  0.00010232123579618389,
  0.00012334614726115318,
  0.00013596109414013474,
  0.00011914116496815932,
  9.11079496815336e-05,
  0.00011773950420382803,
  0.00010372289656051518,
  0.00011213286114650288],
 2: [0.00016966850693710136,
  0.0001806655397941357,
  0.000190091567957308,
  0.000190091567957308,
  0.0002183696524468249,
  0.00020108860081434235,
  0.00018223654448799775,
  0.0001963755867327562,
  0.000190091567957308,
  0.00019166257265117005],
 3: [0.0002578489733847339,
  0.0002303450828903623,
  0.0002114361581754818,
  0.0002578489733847339,
  0.00024065904182575163,
  0.00026816293232012324,
  0.0002561299802288356,
  0.00029050984334680016,
  0.000242378034981649